# How to evaluate a RAG application

This example uses [Langchain](https://www.langchain.com) and [Giskard](https://github.com/Giskard-AI/giskard) to evaluate the quality of a RAG application.
# Interpretation of the Output:
# The following section provides an interpretation of the outputs generated by the code cells. It includes insights and explanations to help understand the results better.


In [1]:
import os
from dotenv import load_dotenv

load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
MODEL = "gpt-3.5-turbo"

## Scrape the Website and Split the Content
# Interpretation of the Output:
# The following section provides an interpretation of the outputs generated by the code cells. It includes insights and explanations to help understand the results better.


In [2]:


from langchain_community.document_loaders import WebBaseLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=20)

loader = WebBaseLoader("https://www.nike.com/gb/w/mens-shoes-nik1zy7ok")
documents = loader.load_and_split(text_splitter)
documents

USER_AGENT environment variable not set, consider setting it to identify your requests.


[Document(page_content="Men's Trainers, Shoes & Sneakers. Nike UK", metadata={'source': 'https://www.nike.com/gb/w/mens-shoes-nik1zy7ok', 'title': "Men's Trainers, Shoes & Sneakers. Nike UK\n", 'description': "Our men's trainers give you the support you need to chase down your dreams. Unleash your potential with Nike. Free Delivery and Returns.", 'language': 'en-GB'}),
 Document(page_content='Skip to main contentFind a StoreHelpHelpOrder StatusDispatch and DeliveryReturnsSize ChartsContact UsPrivacy PolicyTerms of SaleTerms of UseSend Us FeedbackJoin UsSign InNew & FeaturedFeatured Shop All New ArrivalsBest SellersMember Shop \uf8ffüî•SNKRS Launch Calendar National Team Kits 2024Shop IconsAir Force 1Air Jordan 1Air MaxDunkBlazerPegasusMercurialDiscover SportFootballRunningBasketballFitnessGolfTennisYoga DanceSkateboardingTrendingAir Max HomeY2K SneakersNike Style By Jordan Retro CollectionTeensEasyOnNike Gift Ideas SustainabilityMenFeatured New ReleasesBest SellersAir Max HomeY2K Sneak

## Load the Content in a Vector Store
# Interpretation of the Output:
# The following section provides an interpretation of the outputs generated by the code cells. It includes insights and explanations to help understand the results better.


In [3]:


from langchain_openai.embeddings import OpenAIEmbeddings
from langchain_community.vectorstores import DocArrayInMemorySearch

vectorstore = DocArrayInMemorySearch.from_documents(
    documents, embedding=OpenAIEmbeddings()
)

2024-06-30 13:16:10.405039: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


## Create a Knowledge Base
# Interpretation of the Output:
# The following section provides an interpretation of the outputs generated by the code cells. It includes insights and explanations to help understand the results better.


Let's start by loading the content in a pandas DataFrame.
# Interpretation of the Output:
# The following section provides an interpretation of the outputs generated by the code cells. It includes insights and explanations to help understand the results better.


In [4]:


import pandas as pd

df = pd.DataFrame([d.page_content for d in documents], columns=["text"])
df.head(10)

,text
0,"Men's Trainers, Shoes & Sneakers. Nike UK"
1,Skip to main contentFind a StoreHelpHelpOrder ...
2,EquipmentBags and BackpacksHeadwearSocks Women...
3,EquipmentBags and BackpacksHeadwearSocksSaleSa...
4,Trainers for MenLifestyleJordanRunningBasketba...
5,Air Max 95Nike Air Max 95Men's Shoes3 Colours¬...
6,Colours¬£109.99Nike P-6000Just InNike P-6000Sh...
7,guideBest trail-running shoes by NikeBuying gu...
8,shoes that are made to work as hard as you do.
9,Success from the ground up\n\nA great sporting...


We can now create a Knowledge Base using the DataFrame we created before.
# Interpretation of the Output:
# The following section provides an interpretation of the outputs generated by the code cells. It includes insights and explanations to help understand the results better.


In [5]:


from giskard.rag import KnowledgeBase

knowledge_base = KnowledgeBase(df)

/Users/gauravmalhotra/lib/python3.11/site-packages/giskard/core/validation.py:31: RuntimeWarning: fields may not start with an underscore, ignoring "_args"
  validated_func = validate_arguments(func, config={"arbitrary_types_allowed": True})
/Users/gauravmalhotra/lib/python3.11/site-packages/giskard/core/validation.py:31: RuntimeWarning: fields may not start with an underscore, ignoring "_kwargs"
  validated_func = validate_arguments(func, config={"arbitrary_types_allowed": True})
/Users/gauravmalhotra/lib/python3.11/site-packages/threadpoolctl.py:1214: RuntimeWarning: 
Found Intel OpenMP ('libiomp') and LLVM OpenMP ('libomp') loaded at
the same time. Both libraries are known to be incompatible and this
can cause random crashes or deadlocks on Linux when loaded in the
same Python program.
Using threadpoolctl may cause crashes or deadlocks. For more
information and possible workarounds, please see
    https://github.com/joblib/threadpoolctl/blob/master/multiple_openmp.md

  warnings.war

## Generate the Test Set
# Interpretation of the Output:
# The following section provides an interpretation of the outputs generated by the code cells. It includes insights and explanations to help understand the results better.


In [6]:


from giskard.rag import generate_testset

testset = generate_testset(
    knowledge_base,
    num_questions=10,
    agent_description="A chatbot answering questions about the Nike Men's Shoes Website",
)

2024-06-30 13:16:59,218 pid:2510 MainThread giskard.rag  INFO     Finding topics in the knowledge base.


/Users/gauravmalhotra/lib/python3.11/site-packages/umap/umap_.py:2437: UserWarning: n_neighbors is larger than the dataset size; truncating to X.shape[0] - 1
  warn(


2024-06-30 13:17:05,524 pid:2510 MainThread giskard.rag  INFO     Found 1 topics in the knowledge base.


Generating questions:   0%|          | 0/10 [00:00<?, ?it/s]

Let's display a few samples from the test set.
# Interpretation of the Output:
# The following section provides an interpretation of the outputs generated by the code cells. It includes insights and explanations to help understand the results better.


In [7]:


test_set_df = testset.to_pandas()

for index, row in enumerate(test_set_df.head(3).iterrows()):
    print(f"Question {index + 1}: {row[1]['question']}")
    print(f"Reference answer: {row[1]['reference_answer']}")
    print("Reference context:")
    print(row[1]['reference_context'])
    print("******************", end="\n\n")


Question 1: What are the categories of items available for men on the Nike Men's Shoes Website?
Reference answer: The categories of items available for men on the Nike Men's Shoes Website are Shoes, Clothing, and Accessories and Equipment. Within these categories, you can find Lifestyle, Jordan, Running, Football, Basketball, Training and Gym, Skateboarding shoes, Tops and T-Shirts, Hoodies and Sweatshirts, Shorts, Trousers and Tights, Tracksuits, Jackets, Kits and Jerseys clothing, and Bags and Equipment.
Reference context:
Document 2: EquipmentBags and BackpacksHeadwearSocks WomenFeaturedNew ReleasesBest SellersNike Style ByAir Max HomeY2K SneakersNational Team Kits 2024Summer EssentialsShoesAll ShoesLifestyleJordanRunningTraining and GymFootballNike By YouClothingAll ClothingTops and T-ShirtsHoodies and SweatshirtsLeggingsShortsTrousersMatching SetsJacketsSports BrasSkirts and DressesDiscover SportFitnessRunningFootballBasketballTennisDanceYogaGolfAccessories and EquipmentAll Access

Let's now save the test set to a file:
# Interpretation of the Output:
# The following section provides an interpretation of the outputs generated by the code cells. It includes insights and explanations to help understand the results better.


In [8]:
# Detailed Explanation:
# This cell executes the following operations step-by-step:
# testset.save("test-set.jsonl")
# End of detailed explanation.

testset.save("test-set.jsonl")

## Prepare the Prompt Template
# Interpretation of the Output:
# The following section provides an interpretation of the outputs generated by the code cells. It includes insights and explanations to help understand the results better.


In [9]:


from langchain.prompts import PromptTemplate

template = """
Answer the question based on the context below. If you can't 
answer the question, reply "I don't know".

Context: {context}

Question: {question}
"""

prompt = PromptTemplate.from_template(template)
print(prompt.format(context="Here is some context", question="Here is a question"))


Answer the question based on the context below. If you can't 
answer the question, reply "I don't know".

Context: Here is some context

Question: Here is a question



## Create the RAG Chain
# Interpretation of the Output:
# The following section provides an interpretation of the outputs generated by the code cells. It includes insights and explanations to help understand the results better.


Create a retriever from the Vector Store that will allow us to get the top similar documents to a given question.
# Interpretation of the Output:
# The following section provides an interpretation of the outputs generated by the code cells. It includes insights and explanations to help understand the results better.


In [10]:


retriever = vectorstore.as_retriever()
retriever.get_relevant_documents("What is Nike Men's Shoes Website?")

/Users/gauravmalhotra/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


[Document(page_content="Men's Trainers, Shoes & Sneakers. Nike UK", metadata={'source': 'https://www.nike.com/gb/w/mens-shoes-nik1zy7ok', 'title': "Men's Trainers, Shoes & Sneakers. Nike UK\n", 'description': "Our men's trainers give you the support you need to chase down your dreams. Unleash your potential with Nike. Free Delivery and Returns.", 'language': 'en-GB'}),
 Document(page_content="Trainers for MenLifestyleJordanRunningBasketballFootballTraining & GymSkateboardingGolfTennisWalkingGender¬†(1)MenWomenUnisexShop By Price¬†(0)Under ¬£50¬£50 - ¬£100¬£100 - ¬£150Over ¬£150Sale & Offers¬†(0)SaleSize¬†(0)2.533.544.555.56 (EU 39)6 (EU 40)6.577.588.599.51010.51111.51212.51314151617Colour¬†(0)BlackBlueBrownGreenGreyMulti-ColourOrangePinkPurpleRedWhiteYellowShoe Height¬†(0)Low TopMid TopHigh TopCollections¬†(0)Nike MotivaWinfloNike GT SeriesNike Invincible+ MoreFoampositeNike RomaleosAir Force 1Air MaxBlazerCortezEliteHuaracheInternationalistNike AlphaflyNike VaporflyNike Zoom FlyMercur

We can now create our chain.
# Interpretation of the Output:
# The following section provides an interpretation of the outputs generated by the code cells. It includes insights and explanations to help understand the results better.


In [11]:


from langchain_openai.chat_models import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from operator import itemgetter

model = ChatOpenAI(openai_api_key=OPENAI_API_KEY, model=MODEL)

chain = (
    {
        "context": itemgetter("question") | retriever,
        "question": itemgetter("question"),
    }
    | prompt
    | model
    | StrOutputParser()
)

Let's make sure the chain works by testing it with a simple question.
# Interpretation of the Output:
# The following section provides an interpretation of the outputs generated by the code cells. It includes insights and explanations to help understand the results better.


In [12]:


chain.invoke({"question": "What is Nike Men's Shoes Website?"})

"The Nike Men's Shoes website is https://www.nike.com/gb/w/mens-shoes-nik1zy7ok."

## Evaluating the Model on the Test Set
# Interpretation of the Output:
# The following section provides an interpretation of the outputs generated by the code cells. It includes insights and explanations to help understand the results better.


We need to create a function that invokes the chain with a specific question and returns the answer.
# Interpretation of the Output:
# The following section provides an interpretation of the outputs generated by the code cells. It includes insights and explanations to help understand the results better.


In [13]:


def answer_fn(question, history=None):
    return chain.invoke({"question": question})

We can now use the `evaluate()` function to evaluate the model on the test set. This function will compare the answers from the chain with the reference answers in the test set.
# Interpretation of the Output:
# The following section provides an interpretation of the outputs generated by the code cells. It includes insights and explanations to help understand the results better.


In [14]:


from giskard.rag import evaluate

report = evaluate(answer_fn, testset=testset, knowledge_base=knowledge_base)

Asking questions to the agent:   0%|          | 0/10 [00:00<?, ?it/s]

CorrectnessMetric evaluation:   0%|          | 0/10 [00:00<?, ?it/s]

Let now display the report.

Here are the five components of our RAG application:

* **Generator**: This is the LLM used in the chain to generate the answers.
* **Retriever**: This is the retriever that fetches relevant documents from the knowledge base according to a query.
* **Rewriter**: This is a component that rewrites the user query to make it more relevant to the knowledge base or to account for chat history.
* **Router**: This is a component that filters the query of the user based on his intentions.
* **Knowledge Base**: This is the set of documents given to the RAG to generate the answers.
# Interpretation of the Output:
# The following section provides an interpretation of the outputs generated by the code cells. It includes insights and explanations to help understand the results better.


In [15]:


display(report)

Loading BokehJS ...

In [16]:

report.to_html("report.html")

We can display the correctness results organized by question type.
# Interpretation of the Output:
# The following section provides an interpretation of the outputs generated by the code cells. It includes insights and explanations to help understand the results better.


In [17]:


report.correctness_by_question_type()

,correctness
question_type,
complex,0.5
conversational,0.0
distracting element,0.0
double,1.0
simple,0.5
situational,1.0


We can also display the specific failures.
# Interpretation of the Output:
# The following section provides an interpretation of the outputs generated by the code cells. It includes insights and explanations to help understand the results better.


In [18]:


report.get_failures()

,question,reference_answer,reference_context,conversation_history,metadata,agent_answer,correctness,correctness_reason
id,,,,,,,,
5256ce12-fe90-44f0-bd29-7dd215c2f78f,What are the categories of items available for...,The categories of items available for men on t...,Document 2: EquipmentBags and BackpacksHeadwea...,[],"{'question_type': 'simple', 'seed_document_id'...",The categories of items available for men on t...,False,The agent's answer only includes the subcatego...
cff1c122-ce22-4002-a3a8-0b744c334faf,Could you specify the various categories of pr...,The Nike Men's Shoes Website offers various pr...,Document 2: EquipmentBags and BackpacksHeadwea...,[],"{'question_type': 'complex', 'seed_document_id...",The various categories of products available f...,False,"The agent only mentioned shoe categories, whil..."
6bd65c21-6c68-4bd0-976d-438e26adf603,What categories of products are available in t...,The sale section for women on the Nike Men's S...,Document 2: EquipmentBags and BackpacksHeadwea...,[],"{'question_type': 'distracting element', 'seed...",I don't know.,False,The agent failed to provide the correct inform...
6ccbfd67-085b-4021-9a7c-f5fd2861888e,Considering the price range of trainers for me...,The Nike Air Max Plus is priced at £184.99.,Document 6: Colours¬£109.99Nike P-6000Just InN...,[],"{'question_type': 'distracting element', 'seed...",The Nike Air Max Plus falls in the price range...,False,The agent provided an incorrect price for the ...
237eaea5-d226-4d24-821d-1d616b89181e,What are they?,Nike football boots have a variety of features...,Document 11: From after-work kickabouts to Sat...,"[{'role': 'user', 'content': 'I'm interested i...","{'question_type': 'conversational', 'seed_docu...","Men's trainers, shoes, and sneakers.",False,The agent's answer does not match the ground t...
